# 4章コードの補足

In [1]:
import numpy as np
import collections

# negative_sampling_layer
## UnigramSampler
- CBOWモデルの高速化の後半部分．
- 負例をランダムに抽出して学習させる際の選び方．
- コーパス中の単語の出現確率に従ってサンプリングする．

In [13]:
# コーパスは単語IDのリスト．
corpus = np.array([1, 1, 3, 2, 1, 1, 0, 3, 4, 5, 0, 0])

# Counterでコーパス中の単語の出現回数をカウントできる
counts = collections.Counter()
for word_id in corpus:
    counts[word_id] += 1
print(counts)
print(counts[0])

Counter({1: 4, 0: 3, 3: 2, 2: 1, 4: 1, 5: 1})
3


In [21]:
vocab_size = len(counts)  # countsの長さが語彙数
p = np.zeros(vocab_size)  # 語彙数と同じ要素数の出力となる配列を用意
for i in range(vocab_size):
    p[i] = counts[i]  # pのi番目に単語IDがiの出現回数を格納

# 出現回数を0.75乗して稀な単語の確率に少し下駄をはかせる
power = 0.75
word_p = np.power(p, power)
print(f'original_p: {p}')
print(f'word_p: {word_p}')

# np.sum(p) = 全単語数で割って確率にする
p /= np.sum(p)
word_p /= np.sum(word_p)
print(f'p_out: {p}')
print(f'word_p_out: {word_p}')

original_p: [3. 4. 1. 2. 1. 1.]
word_p: [2.27950706 2.82842712 1.         1.68179283 1.         1.        ]
p_out: [0.25       0.33333333 0.08333333 0.16666667 0.08333333 0.08333333]
word_p_out: [0.23284685 0.28891787 0.10214789 0.1717916  0.10214789 0.10214789]
